# Lab 7-1: Tips

Author: Seungjae Lee (이승재)

In [655]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [656]:
# For reproducibility
torch.manual_seed(1)

## Training and Test Datasets

In [657]:
# 학습용 데이터셋
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [658]:
# 테스트용 데이터셋
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

## Model

In [659]:
# 모델 생성
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)
    def forward(self, x):
        return self.linear(x)

In [660]:
model = SoftmaxClassifierModel()

In [661]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [662]:
# 학습 함수 생성
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.cross_entropy(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [663]:
# 테스트 함수. 정확도를 계산한다.
def test(model, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)

    print('Accuracy: {}% Cost: {:.6f}'.format(
         correct_count / len(y_test) * 100, cost.item()
    ))

In [664]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 2.201455
Epoch    2/20 Cost: 2.199247
Epoch    3/20 Cost: 2.197041
Epoch    4/20 Cost: 2.194837
Epoch    5/20 Cost: 2.192636
Epoch    6/20 Cost: 2.190437
Epoch    7/20 Cost: 2.188241
Epoch    8/20 Cost: 2.186049
Epoch    9/20 Cost: 2.183858
Epoch   10/20 Cost: 2.181669
Epoch   11/20 Cost: 2.179484
Epoch   12/20 Cost: 2.177302
Epoch   13/20 Cost: 2.175121
Epoch   14/20 Cost: 2.172944
Epoch   15/20 Cost: 2.170769
Epoch   16/20 Cost: 2.168597
Epoch   17/20 Cost: 2.166427
Epoch   18/20 Cost: 2.164259
Epoch   19/20 Cost: 2.162095


In [665]:
test(model, x_test, y_test)

Accuracy: 33.33333333333333% Cost: 1.218172


## Learning Rate

Gradient Descent 에서의 $\alpha$ 값

`optimizer = optim.SGD(model.parameters(), lr=0.1)` 에서 `lr=0.1` 이다

learning rate이 너무 크면 diverge 하면서 cost 가 점점 늘어난다 (overshooting).

In [666]:
model = SoftmaxClassifierModel()

In [667]:
optimizer = optim.SGD(model.parameters(), lr=1e5) # lr = 학습률

In [668]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.280268
Epoch    1/20 Cost: 976950.687500
Epoch    2/20 Cost: 1279135.125000
Epoch    3/20 Cost: 1198379.125000
Epoch    4/20 Cost: 1098825.625000
Epoch    5/20 Cost: 1968197.625000
Epoch    6/20 Cost: 284763.156250
Epoch    7/20 Cost: 1532260.125000
Epoch    8/20 Cost: 1651504.000000
Epoch    9/20 Cost: 521878.437500
Epoch   10/20 Cost: 1397263.125000
Epoch   11/20 Cost: 750986.250000
Epoch   12/20 Cost: 918691.500000
Epoch   13/20 Cost: 1487888.125000
Epoch   14/20 Cost: 1582260.125000
Epoch   15/20 Cost: 685818.062500
Epoch   16/20 Cost: 1140048.750000
Epoch   17/20 Cost: 940566.562500
Epoch   18/20 Cost: 931638.125000
Epoch   19/20 Cost: 1971322.625000


learning rate이 너무 작으면 cost가 거의 줄어들지 않는다.

In [669]:
model = SoftmaxClassifierModel()

In [670]:
optimizer = optim.SGD(model.parameters(), lr=1e-10) # 학습률이 너무 낮음

In [671]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 3.187324
Epoch    2/20 Cost: 3.187324
Epoch    3/20 Cost: 3.187324
Epoch    4/20 Cost: 3.187324
Epoch    5/20 Cost: 3.187324
Epoch    6/20 Cost: 3.187324
Epoch    7/20 Cost: 3.187324
Epoch    8/20 Cost: 3.187324
Epoch    9/20 Cost: 3.187324
Epoch   10/20 Cost: 3.187324
Epoch   11/20 Cost: 3.187324
Epoch   12/20 Cost: 3.187324
Epoch   13/20 Cost: 3.187324
Epoch   14/20 Cost: 3.187324
Epoch   15/20 Cost: 3.187324
Epoch   16/20 Cost: 3.187324
Epoch   17/20 Cost: 3.187324
Epoch   18/20 Cost: 3.187324
Epoch   19/20 Cost: 3.187324


적절한 숫자로 시작해 발산하면 작게, cost가 줄어들지 않으면 크게 조정하자.

In [672]:
model = SoftmaxClassifierModel()

In [673]:
optimizer = optim.SGD(model.parameters(), lr=1e-4) # 발산하지 않게끔 학습률 조정

In [674]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.341574
Epoch    1/20 Cost: 1.341307
Epoch    2/20 Cost: 1.341040
Epoch    3/20 Cost: 1.340774
Epoch    4/20 Cost: 1.340509
Epoch    5/20 Cost: 1.340244
Epoch    6/20 Cost: 1.339979
Epoch    7/20 Cost: 1.339715
Epoch    8/20 Cost: 1.339452
Epoch    9/20 Cost: 1.339188
Epoch   10/20 Cost: 1.338926
Epoch   11/20 Cost: 1.338663
Epoch   12/20 Cost: 1.338402
Epoch   13/20 Cost: 1.338141
Epoch   14/20 Cost: 1.337880
Epoch   15/20 Cost: 1.337620
Epoch   16/20 Cost: 1.337360
Epoch   17/20 Cost: 1.337101
Epoch   18/20 Cost: 1.336842
Epoch   19/20 Cost: 1.336583


## Data Preprocessing (데이터 전처리)

데이터를 zero-center하고 normalize하자.

In [675]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

$$ x'_j = \frac{x_j - \mu_j}{\sigma_j} $$

여기서 $\sigma$ 는 standard deviation, $\mu$ 는 평균값 이다.

In [676]:
mu = x_train.mean(dim=0) # 평균(x1, x2, x3별 평균)

In [677]:
sigma = x_train.std(dim=0) # x1, x2, x3별 표준편차

In [678]:
norm_x_train = (x_train - mu) / sigma # normalize해줌

In [679]:
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


Normalize와 zero center한 X로 학습해서 성능을 보자

In [680]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [681]:
model = MultivariateLinearRegressionModel()

In [682]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [683]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [684]:
train(model, optimizer, norm_x_train, y_train) # normalize하니까 성능이 더 안좋아짐. 뭐지?

Epoch    0/20 Cost: 29785.089844
Epoch    1/20 Cost: 18906.166016
Epoch    2/20 Cost: 12054.675781
Epoch    3/20 Cost: 7702.029785
Epoch    4/20 Cost: 4925.733398
Epoch    5/20 Cost: 3151.632812
Epoch    6/20 Cost: 2016.996094
Epoch    7/20 Cost: 1291.051270
Epoch    8/20 Cost: 826.505249
Epoch    9/20 Cost: 529.207397
Epoch   10/20 Cost: 338.934174
Epoch   11/20 Cost: 217.153564
Epoch   12/20 Cost: 139.206757
Epoch   13/20 Cost: 89.313782
Epoch   14/20 Cost: 57.375465
Epoch   15/20 Cost: 36.928429
Epoch   16/20 Cost: 23.835773
Epoch   17/20 Cost: 15.450401
Epoch   18/20 Cost: 10.077823
Epoch   19/20 Cost: 6.633700


## Overfitting

너무 학습 데이터에 한해 잘 학습해 테스트 데이터에 좋은 성능을 내지 못할 수도 있다.

이것을 방지하는 방법은 크게 세 가지인데:

1. 더 많은 학습 데이터
2. 더 적은 양의 feature
3. **Regularization**

Regularization: Let's not have too big numbers in the weights

In [685]:
def train_with_regularization(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # l2 norm 계산
        l2_reg = 0
        for param in model.parameters():
            l2_reg += torch.norm(param)
            
        cost += l2_reg # loss에 l2_reg를 더한다. 이는 선형 모델의 일반화 능력을 항상 개선하는 효과가 있다. 

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch+1, nb_epochs, cost.item()
        ))

In [692]:
model = MultivariateLinearRegressionModel()

In [693]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [705]:
train_with_regularization(model, optimizer, norm_x_train, y_train) # 정규화 반영해서 학습. 

Epoch    1/20 Cost: 184.483994
Epoch    2/20 Cost: 184.484009
Epoch    3/20 Cost: 184.483994
Epoch    4/20 Cost: 184.483994
Epoch    5/20 Cost: 184.483994
Epoch    6/20 Cost: 184.483994
Epoch    7/20 Cost: 184.484009
Epoch    8/20 Cost: 184.483994
Epoch    9/20 Cost: 184.483994
Epoch   10/20 Cost: 184.483994
Epoch   11/20 Cost: 184.483994
Epoch   12/20 Cost: 184.483994
Epoch   13/20 Cost: 184.483994
Epoch   14/20 Cost: 184.483994
Epoch   15/20 Cost: 184.483994
Epoch   16/20 Cost: 184.483978
Epoch   17/20 Cost: 184.483978
Epoch   18/20 Cost: 184.483978
Epoch   19/20 Cost: 184.483963
Epoch   20/20 Cost: 184.483978
